In [2]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from collections import OrderedDict
WORKING_DIR = '.'

dataset_educ = pd.read_json(f'{WORKING_DIR}/Dataset_educ_1.1.json', orient = 'index')
dataset_div = pd.read_json(f'{WORKING_DIR}/Dataset_div2_final.json', orient = 'index')


educ_rounds = list(OrderedDict.fromkeys([re.search(r'(blog/entry/[0-9]+)', idx).groups()[0] for idx in dataset_educ.index]))
div_rounds = list(OrderedDict.fromkeys([re.search(r'(blog/entry/[0-9]+)', idx).groups()[0] for idx in dataset_div.index]))

In [3]:
import numpy as np
import random
RANDOM_SEED = 443
DEVICE = "cuda"

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

groups = list(dataset_educ.groupby(lambda k : k.split("?")[0]))
random.shuffle(groups)

train_groups = groups[:10]
validation_groups = groups[10:13]
test_groups = groups[13:]

div_groups = list(dataset_div.groupby(lambda k : k.split("?")[0]))


In [66]:
import re
from sklearn.metrics import classification_report
##Testing educ rounds with reframe
predicted_labels = {}
truth_labels = {}

for idx, round in enumerate(test_groups):
    round_df = round[1]

    for _, comment in round_df.iterrows():
        truth_labels[comment.id] = comment.label

    chatgpt_conversation = ''

    with open(f'conversations/educ/{idx}.txt', 'r', encoding='utf-8', errors='ignore') as fp:
        chatgpt_conversation = fp.read()
        chatgpt_conversation = chatgpt_conversation.split('|^_^|')
    
    #skip first comment
    chatgpt_conversation = chatgpt_conversation[1:]

    for comment in chatgpt_conversation:
        if("prompt: %rule%" in comment):
           response = comment.split("response:")[1]
           
           found = re.findall(r'\(+-*[0-9]+,[\s\(]+([0-9]+)\)+ => (.*)', response)

           for found_id, found_label in found:
               predicted_labels[int(found_id)] = found_label

print(f"Total comments: {len(truth_labels)}")
print(f"Missing comments: {len(set(truth_labels.keys()) - set(predicted_labels.keys()))}")
print(f"Extra comments: {len(set(predicted_labels.keys()) - set(truth_labels.keys()))}")

common_comment_ids = list(set(truth_labels.keys()).intersection(set(predicted_labels.keys())))

common_predicted_labels = [predicted_labels[id] if predicted_labels[id] == 'Irrelevant' else 'Relevant' for id in common_comment_ids]
common_truth_labels = [truth_labels[id] if truth_labels[id] == 'Irrelevant' else 'Relevant' for id in common_comment_ids]

print(classification_report(common_truth_labels, common_predicted_labels))

Total comments: 160
Missing comments: 24
Extra comments: 0
              precision    recall  f1-score   support

  Irrelevant       0.93      0.46      0.62        61
    Relevant       0.69      0.97      0.81        75

    accuracy                           0.74       136
   macro avg       0.81      0.72      0.71       136
weighted avg       0.80      0.74      0.72       136



In [67]:
for idx, round in enumerate(div_groups):
    round_df = round[1]

    for _, comment in round_df.iterrows():
        truth_labels[comment.id] = comment.label

    chatgpt_conversation = ''

    with open(f'conversations/div/{idx}.txt', 'r', encoding='utf-8', errors='ignore') as fp:
        chatgpt_conversation = fp.read()
        chatgpt_conversation = chatgpt_conversation.split('|^_^|')
    
    #skip first comment
    chatgpt_conversation = chatgpt_conversation[1:]

    for comment in chatgpt_conversation:
        if("prompt: %rule%" in comment):
           response = comment.split("response:")[1]
           
           found = re.findall(r'\(+-*[0-9]+,[\s\(]+([0-9]+)\)+ => (.*)', response)

           for found_id, found_label in found:
               predicted_labels[int(found_id)] = found_label

print(f"Total comments: {len(truth_labels)}")
print(f"Missing comments: {len(set(truth_labels.keys()) - set(predicted_labels.keys()))}")
print(f"Extra comments: {len(set(predicted_labels.keys()) - set(truth_labels.keys()))}")

common_comment_ids = list(set(truth_labels.keys()).intersection(set(predicted_labels.keys())))

common_predicted_labels = [predicted_labels[id] if predicted_labels[id] == 'Irrelevant' else 'Relevant' for id in common_comment_ids]
common_truth_labels = [truth_labels[id] if truth_labels[id] == 'Irrelevant' else 'Relevant' for id in common_comment_ids]

print(classification_report(common_truth_labels, common_predicted_labels))

Total comments: 664
Missing comments: 26
Extra comments: 0
              precision    recall  f1-score   support

  Irrelevant       0.97      0.31      0.47       452
    Relevant       0.37      0.98      0.54       186

    accuracy                           0.51       638
   macro avg       0.67      0.65      0.51       638
weighted avg       0.80      0.51      0.49       638

